In [21]:
import pandas as pd 
import numpy as np
from word2number import w2n
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames



In [2]:
df_ejemplo=pd.read_csv("HR RAW DATA.csv",index_col=0)

In [3]:
#Homogeneizar nombre columnas 
nuevas_columnas = {columna: columna.lower() for columna in df_ejemplo.columns}

In [4]:
df_ejemplo.rename(columns=nuevas_columnas, inplace= True)

In [5]:
#Modificamos nombre de la columna para que sea mas intuitivo
df_ejemplo.rename(columns = {"datebirth": "yearbirth"}, inplace = True )

In [6]:
#Eliminamos las siguientes columnas:
#Role Department: Fusiona otras columnas, no le vemos sentdo
#Number Children: no hay valores
#Over 18: columna sin sentido.Esta desactualizada, se puede calcular con otras columnas
#Same As Month Income: Es una columna repetida, una copia de month income
#Employee Count: No tienen sentido. El recuento se puede hacer por filas, o por números de referncia de empleado
#Salary : la eliminamos y creamos una de nuevo con el cálculo de los datos 

In [7]:
df_ejemplo.drop(['roledepartament','numberchildren','over18', 'sameasmonthlyincome','employeecount','salary','standardhours', 'yearsincurrentrole'],axis = 1, inplace=True)

In [8]:
#lista con columnas a limpiar
columnas_limpiar = ['dailyrate','employeenumber','monthlyincome','performancerating','totalworkingyears','worklifebalance','distancefromhome']

In [9]:
#Para columna de Age

def convertir_a_numero(valor):
    if isinstance(valor, int): # Verifica si el valor ya es un entero, si lo es no hace nada
        return valor
    elif isinstance(valor, str): # Intenta convertir el valor a número
        try: 
            numero = int(valor)
            return numero
        except ValueError:
            try: # aquí es donde se usa la función w2n en el try pq si la conversion falla entonces se usará
                numero = w2n.word_to_num(valor)
                return numero
            except ValueError:
                print(f"Error: No se pudo convertir '{valor}' a número.")
                return None
    else:
        print(f"Error: Tipo de dato no soportado para el valor '{valor}'.")
        return None


In [10]:
df_ejemplo['age'] = df_ejemplo['age'].apply(convertir_a_numero) #llamamos a la función

In [11]:
def limpiar_datos (columna):

      try:
          df_ejemplo[columna] =(df_ejemplo[columna].str.replace('$', ''))
    
      except:
          pass
      
      try:
          df_ejemplo[columna] =(df_ejemplo[columna].str.replace(',', '.')) 
    
      except:
          pass
      try:
          
         df_ejemplo[columna] = df_ejemplo[columna].abs()

      except:
          pass
      
      try:
        df_ejemplo[columna]=pd.to_numeric(df_ejemplo[columna]).astype(float)
      except:
        pass
      return df_ejemplo[columna]

In [12]:
#Limpiamos las columnas de numeros para convertirlas a numericas
for i in columnas_limpiar:
   df_ejemplo[i]=limpiar_datos(i)
   print(df_ejemplo[i])
   
 

0        684.0
1        699.0
2        532.0
3        359.0
4       1319.0
         ...  
1609     530.0
1610     805.0
1611     903.0
1612    1229.0
1613     566.0
Name: dailyrate, Length: 1614, dtype: object
0       162.0
1       259.0
2       319.0
3         NaN
4         NaN
        ...  
1609    967.0
1610    972.0
1611      NaN
1612    990.0
1613      NaN
Name: employeenumber, Length: 1614, dtype: float64
0       19537.0
1       19999.0
2       19232.0
3       17169.0
4           NaN
         ...   
1609    10325.0
1610     4447.0
1611        NaN
1612        NaN
1613    10845.0
Name: monthlyincome, Length: 1614, dtype: float64
0       3.0
1       3.0
2       3.0
3       3.0
4       3.0
       ... 
1609    NaN
1610    3.0
1611    3.0
1612    NaN
1613    3.0
Name: performancerating, Length: 1614, dtype: float64
0        NaN
1       34.0
2       22.0
3        NaN
4        NaN
        ... 
1609     NaN
1610     NaN
1611     9.0
1612    12.0
1613     NaN
Name: totalworkingyears, Lengt

In [13]:
#

_____

- Cambiamos de categoria la columna 'género'. Reemplazamos sus valores de 0 y 1 por Male y Famele.

- Homogeneizamos la columna 'maritalstatus'

- Modificamos los valores de la columna  'remotework', 0 y 1, True or False por Yes y No

- Al analizar la columna de employee number, encontramos muchos valores duplicados y la unica diferencia entre estos duplicados eran los valores que se encuentran en la columna 'remotework'. Allí detectamos que la equivalencia era 0 = 'No' y 1 = 'Yes'


In [14]:
df_ejemplo['gender'] = df_ejemplo['gender'].replace({0: 'male', 1: 'female'})
df_ejemplo['gender'].value_counts()

gender
male      971
female    643
Name: count, dtype: int64

In [15]:
df_ejemplo['maritalstatus']= df_ejemplo['maritalstatus'].replace({'divorced':'Divorced','Marreid':'Married'})

In [16]:
df_ejemplo['remotework'].value_counts()

remotework
1        360
True     345
0        309
False    305
Yes      295
Name: count, dtype: int64

In [27]:
df_ejemplo['remotework']= df_ejemplo['remotework'].replace({('1'):'Yes',('0') or ('False'): 'No', 'True': 'Yes', 'False': 'No'})

In [29]:
df_ejemplo['remotework'].value_counts()

remotework
Yes    1000
No      614
Name: count, dtype: int64

______

In [22]:
df_ejemplo.head(5)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,yearbirth,remotework
0,51,No,NaN,684.0,NaN,6.0,3,NaN,162.0,1,male,51,3,5,resEArch DIREcToR,3,NaN,19537.0,6462,7,No,13,3.0,3,NaN,0,NaN,5,3.0,20,NaN,15,15,1972,Yes
1,52,No,NaN,699.0,NaN,1.0,4,Life Sciences,259.0,3,male,65,2,5,ManAGeR,3,NaN,19999.0,5678,0,NaN,14,3.0,1,NaN,1,34.0,5,3.0,33,NaN,11,9,1971,Yes
2,42,No,travel_rarely,532.0,Research & Development,4.0,2,Technical Degree,319.0,3,male,58,3,5,ManaGER,4,Married,19232.0,4933,1,No,11,3.0,4,NaN,0,22.0,3,NaN,22,NaN,11,15,1981,Yes
3,47,No,travel_rarely,359.0,NaN,2.0,4,Medical,NaN,1,female,82,3,4,ReseArCH DIrECtOr,3,Married,17169.0,26703,3,NaN,19,3.0,2,NaN,2,NaN,2,NaN,20,NaN,5,6,1976,False
4,46,No,NaN,1319.0,NaN,3.0,3,Technical Degree,NaN,1,female,45,4,4,sAleS EXECUtIve,1,Divorced,NaN,7739,2,No,12,3.0,4,NaN,1,NaN,5,3.0,19,NaN,2,8,1977,No


Homogeneizamos las categorias de las encuestas (que todas vayan de 1 a 4)

Modificamos los valores de Education para que sean mas legibles
Cambiariamos los float a int. Lo miramos cuando gestionamos nulos

JobLevel, esta de 1 a 5. Miramos de cambiar por categorias 
Para el markdown:
Entry:
Limited or prior experience
Non Exempt
PSS
Intermediate:
Assignments of moderate scope and complexity
Judgment within defined guidelines
Non Exempt
PSS
Experienced:
In depth understanding of field
Independently performs full range of function
Exempt
PSS
Advanced:
High degree knowledge of overall field
Specialized knowledge
High degree of autonomy
Lead
Exempt
PSS or MSP
Expert:
Significant impact on policy program
Substantial consequences of success or failure
Scope impacts entire organization (campus)
Exempt
MSP


In [56]:
df_ejemplo['jobsatisfaction'].value_counts() #1 a 4
df_ejemplo['stockoptionlevel'].value_counts() # 0 a 3
df_ejemplo['relationshipsatisfaction'].value_counts() # 1 a 4
df_ejemplo['performancerating'].value_counts() # 3.0 y 4.0 
df_ejemplo['jobinvolvement'].value_counts() #1 a 4
df_ejemplo['worklifebalance'].value_counts() #1.0 a 4.0
df_ejemplo['trainingtimeslastyear'].value_counts() # 0 a 6
df_ejemplo['joblevel'].value_counts() # 1 a 5


joblevel
2    597
1    586
3    242
4    113
5     76
Name: count, dtype: int64

In [53]:
datos = {1: 'High school', 2: 'College', 3: 'University', 4:'Master Degree', 5: 'PhD'}

In [54]:
df_ejemplo['education']= df_ejemplo['education'].replace(datos)

In [58]:
categorias = {1: 'Entry', 2: 'Intermediate', 3: 'Experienced', 4: 'Advanced', 5: 'Expert'}

In [59]:
df_ejemplo['joblevel']= df_ejemplo['joblevel'].replace(categorias)

______

In [66]:
df_ejemplo.to_csv('hr_limpio_con_nulos.csv')